In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
df = pd.read_csv("/content/drive/My Drive/Projetos/Rec_carros/Scraper_carro/Data/data_geral_rotulada.csv", sep=';')
df.set_index('id', inplace= True)
df

,modelo,y,ano,valor,cor,km,opcionais,link
id,,,,,,,,
1014060,c3-hatch-attraction-1-2-12v,1,17/18,"R$43.800,00",Branca,38.000,"['Air Bag Duplo', 'Alarme', 'Ar Condicionado',...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1003530,c3-hatch-attraction-1-2-12v,1,18/19,"R$39.500,00",Marrom,24.536,"['Air Bag Duplo', 'Alarme', 'Ar Condicionado',...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1002771,c3-hatch-attraction-1-5,0,15/15,"R$29.900,00",Branca,47.000,"['Air Bag', 'Air Bag Duplo', 'Alarme', 'Ar Con...",https://www.shopcar.com.br/veiculos/citro-n/c3...
981268,c3-hatch-attraction-1-5,0,14/15,"R$29.900,00",Branca,0.000,"['Air Bag', 'Alarme', 'Ar Condicionado', 'Ar Q...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1002011,c3-hatch-attraction-1-6-16v,0,19/20,"R$62.990,00",Prata,0.000,"['Air Bag Duplo', 'Ar Condicionado', 'Ar Digit...",https://www.shopcar.com.br/veiculos/citro-n/c3...
...,...,...,...,...,...,...,...,...
930084,polo-hatch-1-6-16v,0,20/20,0,Branca,0.000,"['Air Bag', 'Air Bag Duplo', 'Alarme', 'Ar Con...",https://www.shopcar.com.br/veiculos/vw-volkswa...
1002539,polo-hatch-comfortline-1-0-12v-tsi,0,20/20,0,Branca,0.000,"['Air Bag', 'Air Bag Duplo', 'Alarme', 'Ar Con...",https://www.shopcar.com.br/veiculos/vw-volkswa...
996106,polo-hatch-comfortline-1-0-12v-tsi,0,18/19,0,Branca,32.800,"['Air Bag', 'Air Bag Duplo', 'Alarme', 'Ar Con...",https://www.shopcar.com.br/veiculos/vw-volkswa...


# Tratando os dados

## Criando um DF somente com as features

In [3]:
df_features = pd.DataFrame(index=df.index)
df_features.head()

""
id
1014060
1003530
1002771
981268
1002011


## Tratando a coluna Valor

In [4]:
def trat_valor(valor):
  valor = valor.replace("R$", "")
  valor = valor.replace(",", "")
  valor = pd.to_numeric(valor)
  valor = valor*1000
  return valor

In [5]:
df["valor"] = df["valor"].map(trat_valor)
df.head()

,modelo,y,ano,valor,cor,km,opcionais,link
id,,,,,,,,
1014060,c3-hatch-attraction-1-2-12v,1,17/18,43800.0,Branca,38.000,"['Air Bag Duplo', 'Alarme', 'Ar Condicionado',...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1003530,c3-hatch-attraction-1-2-12v,1,18/19,39500.0,Marrom,24.536,"['Air Bag Duplo', 'Alarme', 'Ar Condicionado',...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1002771,c3-hatch-attraction-1-5,0,15/15,29900.0,Branca,47.000,"['Air Bag', 'Air Bag Duplo', 'Alarme', 'Ar Con...",https://www.shopcar.com.br/veiculos/citro-n/c3...
981268,c3-hatch-attraction-1-5,0,14/15,29900.0,Branca,0.000,"['Air Bag', 'Alarme', 'Ar Condicionado', 'Ar Q...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1002011,c3-hatch-attraction-1-6-16v,0,19/20,62990.0,Prata,0.000,"['Air Bag Duplo', 'Ar Condicionado', 'Ar Digit...",https://www.shopcar.com.br/veiculos/citro-n/c3...


In [6]:
df_features["valor"] = df["valor"]
df_features.head()

,valor
id,
1014060,43800.0
1003530,39500.0
1002771,29900.0
981268,29900.0
1002011,62990.0


## Tratando a coluna KM

In [7]:
df["km"] = df["km"]*1000
df.head()

,modelo,y,ano,valor,cor,km,opcionais,link
id,,,,,,,,
1014060,c3-hatch-attraction-1-2-12v,1,17/18,43800.0,Branca,38000.0,"['Air Bag Duplo', 'Alarme', 'Ar Condicionado',...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1003530,c3-hatch-attraction-1-2-12v,1,18/19,39500.0,Marrom,24536.0,"['Air Bag Duplo', 'Alarme', 'Ar Condicionado',...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1002771,c3-hatch-attraction-1-5,0,15/15,29900.0,Branca,47000.0,"['Air Bag', 'Air Bag Duplo', 'Alarme', 'Ar Con...",https://www.shopcar.com.br/veiculos/citro-n/c3...
981268,c3-hatch-attraction-1-5,0,14/15,29900.0,Branca,0.0,"['Air Bag', 'Alarme', 'Ar Condicionado', 'Ar Q...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1002011,c3-hatch-attraction-1-6-16v,0,19/20,62990.0,Prata,0.0,"['Air Bag Duplo', 'Ar Condicionado', 'Ar Digit...",https://www.shopcar.com.br/veiculos/citro-n/c3...


In [8]:
df_features["km"] = df["km"]
df_features.head()

,valor,km
id,,
1014060,43800.0,38000.0
1003530,39500.0,24536.0
1002771,29900.0,47000.0
981268,29900.0,0.0
1002011,62990.0,0.0


## Tratando a coluna Modelo

In [9]:
df['modelo'] = df['modelo'].map(lambda x: x.replace("-", " "))
df.head()

,modelo,y,ano,valor,cor,km,opcionais,link
id,,,,,,,,
1014060,c3 hatch attraction 1 2 12v,1,17/18,43800.0,Branca,38000.0,"['Air Bag Duplo', 'Alarme', 'Ar Condicionado',...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1003530,c3 hatch attraction 1 2 12v,1,18/19,39500.0,Marrom,24536.0,"['Air Bag Duplo', 'Alarme', 'Ar Condicionado',...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1002771,c3 hatch attraction 1 5,0,15/15,29900.0,Branca,47000.0,"['Air Bag', 'Air Bag Duplo', 'Alarme', 'Ar Con...",https://www.shopcar.com.br/veiculos/citro-n/c3...
981268,c3 hatch attraction 1 5,0,14/15,29900.0,Branca,0.0,"['Air Bag', 'Alarme', 'Ar Condicionado', 'Ar Q...",https://www.shopcar.com.br/veiculos/citro-n/c3...
1002011,c3 hatch attraction 1 6 16v,0,19/20,62990.0,Prata,0.0,"['Air Bag Duplo', 'Ar Condicionado', 'Ar Digit...",https://www.shopcar.com.br/veiculos/citro-n/c3...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack, vstack

In [34]:
X= df['modelo']

modelo_train, modelo_valid = train_test_split(X, test_size=0.5, random_state = 2)

modelo_vec = TfidfVectorizer(min_df=2)
modelo_bow_train = modelo_vec.fit_transform(modelo_train)
modelo_bow_valid = modelo_vec.transform(modelo_valid)
modelo_bow_train

<315x52 sparse matrix of type '<class 'numpy.float64'>'
	with 1121 stored elements in Compressed Sparse Row format>

## Tratando a coluna Ano

In [35]:
X= df['ano']

ano_train, ano_valid = train_test_split(X, test_size=0.5, random_state = 2)

ano_vec = TfidfVectorizer(min_df=2)
ano_bow_train = ano_vec.fit_transform(ano_train)
ano_bow_valid = ano_vec.transform(ano_valid)
ano_bow_train

<315x14 sparse matrix of type '<class 'numpy.float64'>'
	with 515 stored elements in Compressed Sparse Row format>

##Tratando a coluna Cor

In [13]:
X= df['cor']

cor_train, cor_valid = train_test_split(X, test_size=0.5, random_state = 0)

cor_vec = TfidfVectorizer(min_df=2)
cor_bow_train = cor_vec.fit_transform(cor_train)
cor_bow_valid = cor_vec.transform(cor_valid)
cor_bow_train

<315x8 sparse matrix of type '<class 'numpy.float64'>'
	with 315 stored elements in Compressed Sparse Row format>

##Tratando a coluna Opcionais

In [14]:
def trat_opcionais(valor):
  valor = valor.replace(" ", "")
  valor = valor.replace("'", "")
  valor = valor.replace(",", " ")
  valor = valor.replace("]", "")
  valor = valor.replace("[", "")
  return valor

In [15]:
df["opcionais"] = df["opcionais"].map(trat_opcionais)
df.head()

,modelo,y,ano,valor,cor,km,opcionais,link
id,,,,,,,,
1014060,c3 hatch attraction 1 2 12v,1,17/18,43800.0,Branca,38000.0,AirBagDuplo Alarme ArCondicionado ArDigital Câ...,https://www.shopcar.com.br/veiculos/citro-n/c3...
1003530,c3 hatch attraction 1 2 12v,1,18/19,39500.0,Marrom,24536.0,AirBagDuplo Alarme ArCondicionado ArQuente Com...,https://www.shopcar.com.br/veiculos/citro-n/c3...
1002771,c3 hatch attraction 1 5,0,15/15,29900.0,Branca,47000.0,AirBag AirBagDuplo Alarme ArCondicionado ArQue...,https://www.shopcar.com.br/veiculos/citro-n/c3...
981268,c3 hatch attraction 1 5,0,14/15,29900.0,Branca,0.0,AirBag Alarme ArCondicionado ArQuente Computad...,https://www.shopcar.com.br/veiculos/citro-n/c3...
1002011,c3 hatch attraction 1 6 16v,0,19/20,62990.0,Prata,0.0,AirBagDuplo ArCondicionado ArDigital ArQuente ...,https://www.shopcar.com.br/veiculos/citro-n/c3...


In [16]:
X= df['opcionais']

opcionais_train, opcionais_valid = train_test_split(X, test_size=0.5, random_state = 0)

opcionais_vec = TfidfVectorizer(max_df=10)
opcionais_bow_train = opcionais_vec.fit_transform(opcionais_train)
opcionais_bow_valid = opcionais_vec.transform(opcionais_valid)
opcionais_bow_train

<315x12 sparse matrix of type '<class 'numpy.float64'>'
	with 51 stored elements in Compressed Sparse Row format>

# Modelando

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

In [36]:
X= df_features
y= df['y']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.5, random_state = 2)

In [37]:
X_train_wbows = hstack([X_train, 
                        modelo_bow_train, 
                        ano_bow_train, 
                        #cor_bow_train, 
                        #opcionais_bow_train
                        ])
X_valid_wbows = hstack([X_valid, 
                        modelo_bow_valid, 
                        ano_bow_valid, 
                        #cor_bow_valid, 
                        #opcionais_bow_valid
                        ])

In [38]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=-1)
mdl.fit(X_train_wbows, y_train)

p = mdl.predict_proba(X_valid_wbows)[:, 1]

print(average_precision_score(y_valid, p))
print(roc_auc_score(y_valid, p))

0.7406779434557214
0.9797044519266741


#Resultado testes

Valor base somente com as features numéricas: </br>
ap - 0.4172000468181098 auc - 0.9300689655172414

## Features: Valor, KM e Modelo

ap - 0.7149813286432144 auc - 0.9473103448275861 - min_df=1 </br>

ap - 0.7387583849379906 auc - 0.9558620689655173 - min_df=2 </br>

ap - 0.7529606294457390 auc - 0.9600689655172410 - min_df=3* </br>

ap - 0.7371485237573607 auc - 0.9593103448275863 - min_df=4 </br>


## Features: Valor, KM, Modelo e Ano

ap - 0.7936754359143990 auc - 0.9784827586206897 - min_df=1* </br>

ap - 0.8029544975114297 auc - 0.9800000000000000 - min_df=2 </br>

ap - 0.7993965226928115 auc - 0.9795172413793103 - min_df=3 </br>

ap - 0.8040066981178436 auc - 0.9795862068965516 - min_df=4 </br>


## Features: Valor, KM, Modelo, Ano e Cor

ap - 0.7967949832808797 auc - 0.9782758620689656 - min_df=1 </br>

ap - 0.7967949832808797 auc - 0.9782758620689656 - min_df=2 </br>

ap - 0.7923105509169762 auc - 0.9776551724137931 - min_df=3 </br>

ap - 0.7890401299219286 auc - 0.9768275862068966 - min_df=4 </br>

## Features: Valor, KM, Modelo, Ano, Cor e Opcionais

ap - 0.3727065991560107 auc - 0.8591034482758620 - min_df=1 </br>

ap - 0.3726404509630315 auc - 0.8599310344827586 - min_df=3 </br>

ap - 0.7803044992003083 auc - 0.9741379310344828 - max_df=20 </br>

##Conclusão = Tunar o min_df aparentemente impacta positivamente somente a feature Modelo. A feature Ano é relevante, aumentando a averege precision em 5%. A feature Cor se mostrou irrelevante ou prejudicial, tendo efeitos positivos mínimos ou levemente negativos. E a feature Opcionais diminiu as métricas abaixo da base e no máximo deixou próximo de quando foi adicionado o Ano.

# Testes

In [21]:
df_teste = pd.DataFrame(index=X_valid.index)

In [22]:
df_teste["p"] = p

In [23]:
df_teste['y'] = y_valid

In [24]:
df_teste['valor'] = X_valid["valor"]

In [25]:
mask = (df_teste["y"] == 1)

In [26]:
mask1 = (df_teste["p"] >= .35)

In [27]:
df_teste[mask1].sort_values("p")

,p,y,valor
id,,,
977038,0.362,0,44900.0
1009901,0.367,1,48500.0
976015,0.403,0,48700.0
1007067,0.405,0,41990.0
1014060,0.419,1,43800.0
1014696,0.475,0,39990.0
1015827,0.510,0,51900.0
1015064,0.538,1,47500.0
1003575,0.632,1,44900.0


#Exportando os dados limpos

In [28]:
df.to_csv("/content/drive/My Drive/Projetos/Rec_carros/clean_data.csv")